In [1]:
import sys
sys.path.append("../")
import rms_torch 
import numpy as np
import torch

In [2]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
Sz = np.zeros([3, 3])
Sz[0, 0] = 1
Sz[2, 2] = -1
Sx = np.zeros([3, 3])
Sx[1, 0] = Sx[0, 1] = Sx[2, 1] = Sx[1, 2] = 1 / np.sqrt(2)
Sy = np.zeros([3, 3], dtype=np.complex64)
Sy[1, 0] = Sy[2, 1] = 1j / np.sqrt(2)
Sy[0, 1] = Sy[1, 2] = -1j / np.sqrt(2)


SzSz = np.kron(Sz, Sz).astype(np.float64)
SxSx = np.kron(Sx, Sx).astype(np.float64)
SySy = np.kron(Sy, Sy).astype(np.float64)
lh = SzSz + SxSx + SySy
lh = lh  + lh @ lh * 1/3

u = np.array(
[
    [-1/np.sqrt(2), 0, 1/np.sqrt(2)],
    [1/np.sqrt(2), 0, 1/np.sqrt(2)],
    [0,1,0]
]
)
U = np.kron(u,u)

lh_u = (U @ lh @ U.conj().T).real

/var/folders/z7/m5qzz0hj12z06hxsn5km2c400000gn/T/ipykernel_53701/1414407092.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  SySy = np.kron(Sy, Sy).astype(np.float64)


In [61]:
L = 4
e0 = L * -2/3
bonds = [[i, (i+1)%L] for i in range(L)]

In [62]:
H = rms_torch.sum_ham(lh, bonds, L, 3)
H_u = rms_torch.sum_ham(lh_u, bonds, L, 3)
H_u_s = rms_torch.stoquastic(H_u)
H_tensor = torch.tensor(H.astype(np.float32), device=mps_device)

In [63]:
e, v = np.linalg.eigh(H)
es, vs = np.linalg.eigh(H_u_s)
eu, vu = np.linalg.eigh(H_u)

In [64]:
print("e0 : ", e0)
print("e[0], es[0], eu[0] : ", e[0], es[0], eu[0])

e0 :  -2.6666666666666665
e[0], es[0], eu[0] :  -2.6666666742355103 -2.6666666742355076 -2.6666666742355063


In [65]:
v0 = v[:,0]
vs0 = vs[:,0]
vu0 = vu[:,0]

In [66]:
e[0], es[0]

(-2.6666666742355103, -2.6666666742355076)

In [67]:
vs0_a = np.abs(vs0)
vs0_a @ H_u_s @ vs0_a

-2.6666666742355054

In [68]:
es

array([-2.66666667, -2.66666667, -2.66666667, -2.66666666, -1.0971676 ,
       -1.09716751, -0.66666673, -0.66666673, -0.6666666 , -0.25170115,
       -0.25170115, -0.25170111, -0.25170111, -0.25170111, -0.25170111,
        0.17863275,  0.17863275,  0.17863275,  0.17863275,  0.17863275,
        0.17863275,  0.66666659,  0.66666659,  0.6666666 ,  0.66666669,
        1.33333316,  1.33333317,  1.33333317,  1.33333317,  1.33333317,
        1.33333321,  1.33333321,  1.33333333,  1.33333333,  1.33333333,
        1.33333333,  1.33333333,  1.33333334,  1.33333334,  1.33333335,
        1.33333335,  1.88862974,  1.88862974,  1.88862984,  1.88862984,
        1.88862984,  1.88862984,  1.99999991,  2.43050073,  2.43050082,
        2.48803377,  2.48803377,  2.4880338 ,  2.4880338 ,  2.4880338 ,
        2.4880338 ,  2.66666657,  2.66666657,  2.66666659,  2.66666659,
        2.66666659,  3.33333324,  3.33333324,  3.99999983,  3.99999983,
        3.99999988,  3.99999988,  3.99999991,  4.00000002,  4.36

In [69]:
print("abs of rho must be rho_sa",np.linalg.norm(np.abs(vu0) - vs0_a))

abs of rho must be rho_sa 2.816998853304935e-08


In [70]:
rho = vu0[:, None] @ vu0[None, :]
rho_sa = vs0_a[:,None] @ vs0_a[None,:]

In [71]:
print("abs of rho must be rho_sa",np.linalg.norm(np.abs(rho) - rho_sa))

abs of rho must be rho_sa 3.9838379835332935e-08


### Abs of partial rho must be prho_abs

In [72]:
rho_ = rho.reshape(9, 3**(L-2), 9, 3**(L-2))
prho = np.einsum("jiki->jk", rho_)
rho_sa_ = rho_sa.reshape(9, 3**(L-2), 9, 3**(L-2))
prho_sa = np.einsum("jiki->jk", rho_sa_)

In [73]:
print("abs of prho must be pro_sa",np.linalg.norm(np.abs(prho) - prho_sa))

abs of prho must be pro_sa 2.843700879150999e-08


In [74]:
S = np.sign(prho)

In [75]:
S

array([[ 1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.],
       [-1.,  1.,  1., -1.,  1.,  1., -1., -1., -1.],
       [ 1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.],
       [ 1., -1., -1.,  1., -1., -1., -1., -1.,  1.],
       [-1.,  1.,  1., -1.,  1., -1., -1.,  1., -1.],
       [ 1.,  1.,  1., -1., -1.,  1., -1., -1.,  1.],
       [-1., -1., -1., -1., -1., -1.,  1., -1.,  1.],
       [-1., -1.,  1., -1.,  1., -1., -1.,  1., -1.],
       [ 1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.]])

In [77]:
np.round(prho,8)

array([[ 0.14285714, -0.        ,  0.        ,  0.        , -0.14285715,
         0.        , -0.        , -0.        ,  0.14285714],
       [-0.        ,  0.0952381 ,  0.        , -0.0952381 ,  0.        ,
         0.        , -0.        , -0.        , -0.        ],
       [ 0.        ,  0.        ,  0.09523809, -0.        ,  0.        ,
         0.        , -0.09523809,  0.        , -0.        ],
       [ 0.        , -0.0952381 , -0.        ,  0.0952381 , -0.        ,
        -0.        , -0.        , -0.        ,  0.        ],
       [-0.14285715,  0.        ,  0.        , -0.        ,  0.14285715,
        -0.        , -0.        ,  0.        , -0.14285715],
       [ 0.        ,  0.        ,  0.        , -0.        , -0.        ,
         0.0952381 , -0.        , -0.0952381 ,  0.        ],
       [-0.        , -0.        , -0.09523809, -0.        , -0.        ,
        -0.        ,  0.09523809, -0.        ,  0.        ],
       [-0.        , -0.        ,  0.        , -0.        ,  0